In [1]:
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline
import numpy as np
from numpy.linalg import svd
from sklearn.decomposition import NMF
from numpy import array, matrix, linalg

In [35]:
df1 = pd.read_csv('StateNames.csv')

In [36]:
def clean_data(df, pivot_vals, pivot_index, pivot_col):
    df['new_name'] = df.Name.str.lower()
    df = df.set_index(['Year','State'])
    df['name_gender'] = zip(df.new_name, df.Gender)
    df = df.drop(['Year', 'State', 'new_name'])
    
    date_state_series = df.groupby(df.index).sum()['Count']
    df['tuple_index'] = df.index.values
    df = df.join(date_state_series, how='left', on='tuple_index', lsuffix='_name_count')
    
    df['count_norm'] = df.Count_name_count / df.Count
    
    data_pivot = pd.pivot_table(df, values=pivot_vals, index=pivot_index, columns=pivot_col)
    data_pivot = data_pivot.fillna(0)
    
    return data_pivot

In [4]:
df = clean_data(df)

In [5]:
date_state_series = df.groupby(df.index).sum()['Count']
# total_dict = date_state_series.to_dict()

In [6]:
#Further cleaning
df['tuple_index'] = df.index.values
df = df.join(date_state_series, how='left', on='tuple_index', lsuffix='name_count')

In [7]:
df['count_norm'] = df.Countname_count / df.Count

In [11]:
name_pivot = pd.pivot_table(df, values='count_norm',index=df.name_gender, columns=df.index)
name_pivot = name_pivot.fillna(0)

In [12]:
name_pivot.shape

(33170, 5355)

In [15]:
def run_nmf(df,params):
    nmf = NMF(n_components=params)
    W = nmf.fit_transform(df)
    H = nmf.components_
    return W, H

In [16]:
W, H = run_nmf(name_pivot,10)

In [28]:
def print_top_features(H, features, top_features):
    for latent_num, latent in enumerate(H):
        print "Latent Feature %d" % (int(latent_num) + 1)
        print [features[i] for i in latent.argsort()[:-top_features-1:-1]]
        print 

In [29]:
state_features = name_pivot.columns

In [32]:
# print_top_features(H, state_features, 15)

In [ ]:
##RUN THIS CODE AGAIN, didn't make the right pivot earlier

In [37]:
state_pivot = clean_data(df1,'count_norm', df.name_gender, df.index)

In [41]:
name_features = state_pivot.columns

In [38]:
W2, H2 = run_nmf(state_pivot,10)

In [42]:
print_top_features(H2, name_features, 15)

Latent Feature 1
[(1942, 'RI'), (1939, 'RI'), (1941, 'RI'), (1940, 'RI'), (1940, 'CT'), (1943, 'RI'), (1939, 'CT'), (1940, 'MA'), (1938, 'RI'), (1939, 'MA'), (1939, 'NH'), (1937, 'RI'), (1941, 'CT'), (1941, 'MA'), (1944, 'RI')]

Latent Feature 2
[(1987, 'ND'), (1987, 'VT'), (1988, 'ND'), (1987, 'ME'), (1988, 'VT'), (1987, 'WY'), (1988, 'ME'), (1988, 'WY'), (1990, 'VT'), (1989, 'VT'), (1989, 'WV'), (1987, 'SD'), (1990, 'WV'), (1989, 'ME'), (1988, 'WV')]

Latent Feature 3
[(1913, 'AK'), (1911, 'AK'), (1910, 'NV'), (1911, 'NV'), (1912, 'NV'), (1912, 'AK'), (1910, 'AK'), (1917, 'AK'), (1918, 'AK'), (1913, 'NV'), (1921, 'AK'), (1916, 'AK'), (1931, 'AK'), (1915, 'AK'), (1924, 'AK')]

Latent Feature 4
[(2008, 'VT'), (2007, 'VT'), (2006, 'VT'), (2009, 'VT'), (2005, 'VT'), (2004, 'VT'), (2003, 'VT'), (2010, 'VT'), (2003, 'ND'), (2011, 'VT'), (2004, 'ND'), (2012, 'VT'), (2006, 'ND'), (2002, 'VT'), (2006, 'ME')]

Latent Feature 5
[(1974, 'RI'), (1973, 'RI'), (1972, 'NH'), (1973, 'VT'), (1972, 'RI